In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, countDistinct, avg, min, max, stddev, desc

spark = SparkSession.builder \
    .appName("Spotify Recommandation") \
    .getOrCreate()

df = spark.read.option("header", "true").option("inferSchema", "true").csv("hdfs://hadoop-namenode-1:8020/data/")

In [2]:
df.printSchema()
# df.show(5, truncate=False)

root
 |-- valence: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)



In [3]:
print("Nombre de lignes :", df.count())

Nombre de lignes : 170653


In [4]:
print("Nombre de colonnes :", len(df.columns))

Nombre de colonnes : 19


In [5]:
df.select("artists").agg(countDistinct("artists")).show()

+-----------------------+
|count(DISTINCT artists)|
+-----------------------+
|                  34017|
+-----------------------+



In [6]:
df.groupBy("year").count().orderBy("year").show(20)

+----+-----+
|year|count|
+----+-----+
|1921|  150|
|1922|   71|
|1923|  185|
|1924|  236|
|1925|  278|
|1926| 1378|
|1927|  615|
|1928| 1261|
|1929|  952|
|1930| 1924|
|1931|  966|
|1932|  502|
|1933|  691|
|1934|  578|
|1935| 1518|
|1936| 1100|
|1937|  657|
|1938|  613|
|1939| 1000|
|1940| 2000|
+----+-----+
only showing top 20 rows



In [7]:
df.select(
    "danceability", "energy", "valence"
).describe().show()

+-------+--------------------+--------------------+--------------------+
|summary|        danceability|              energy|             valence|
+-------+--------------------+--------------------+--------------------+
|  count|              170653|              170653|              170653|
|   mean|  0.5376399623706486|   533.2726662178331|  0.5288041167238217|
| stddev| 0.17595879531415606|  13500.997419182748|  0.2630074934143846|
|    min| ""Kamehameha Sch...| ""Loughton High ...|                 0.0|
|    max|               0.988|              974773|6.409999999999999...|
+-------+--------------------+--------------------+--------------------+



In [8]:
df.select(
    "tempo", "loudness", "duration_ms"
).describe().show()

+-------+--------------------+-------------------+--------------------+
|summary|               tempo|           loudness|         duration_ms|
+-------+--------------------+-------------------+--------------------+
|  count|              170653|             170653|              170653|
|   mean|  118.22609112814219|-11.417017795610022|   230404.3580315134|
| stddev|   67.00273009940202| 5.7298266024160736|  126366.51213915045|
|    min|    'José Feliciano'|   'David Baxter']"| ""'Legally Blond...|
|    max|When Your Lover H...|                  9|               99997|
+-------+--------------------+-------------------+--------------------+



In [9]:
df.select("popularity").groupBy("popularity").count().orderBy(desc("count")).show(10)

+----------+-----+
|popularity|count|
+----------+-----+
|         0|27222|
|        43| 3118|
|        44| 3110|
|        41| 3067|
|        40| 3043|
|        42| 3038|
|        39| 2964|
|         1| 2921|
|        36| 2879|
|        46| 2878|
+----------+-----+
only showing top 10 rows



In [10]:
# Moyenne de popularité par année
df.groupBy("year").agg(avg("popularity").alias("popularité_moyenne")).orderBy("year").show(20)

+----+-------------------+
|year| popularité_moyenne|
+----+-------------------+
|1921| 0.6533333333333333|
|1922|0.14084507042253522|
|1923|  5.396739130434782|
|1924| 0.6610169491525424|
|1925| 2.6137184115523464|
|1926| 1.4223512336719883|
|1927|  0.797716150081566|
|1928| 1.5294117647058822|
|1929| 0.3403361344537815|
|1930| 0.9271970878835153|
|1931|0.17780172413793102|
|1932|  2.151394422310757|
|1933|  6.898697539797395|
|1934| 1.2630662020905923|
|1935| 1.5274798927613942|
|1936|  5.082916058394161|
|1937|  3.328767123287671|
|1938|  2.096247960848287|
|1939|               4.36|
|1940| 0.9332664325137983|
+----+-------------------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import sum as spark_sum

df.select([spark_sum(col(c).isNull().cast("int")).alias(c + "_nulls") for c in df.columns]).show()

+-------------+----------+------------------+-------------+------------------+-----------------+------------+--------------+--------+----------------------+---------+--------------+--------------+----------+----------+----------------+------------------+-----------------+-----------+
|valence_nulls|year_nulls|acousticness_nulls|artists_nulls|danceability_nulls|duration_ms_nulls|energy_nulls|explicit_nulls|id_nulls|instrumentalness_nulls|key_nulls|liveness_nulls|loudness_nulls|mode_nulls|name_nulls|popularity_nulls|release_date_nulls|speechiness_nulls|tempo_nulls|
+-------------+----------+------------------+-------------+------------------+-----------------+------------+--------------+--------+----------------------+---------+--------------+--------------+----------+----------+----------------+------------------+-----------------+-----------+
|            0|         0|                 0|            0|                 0|                0|           0|             0|       0|            

In [13]:
audio_features = [
    "danceability", "energy", "valence", "acousticness"
]

df.select(audio_features).describe().show()

+-------+--------------------+--------------------+--------------------+-------------------+
|summary|        danceability|              energy|             valence|       acousticness|
+-------+--------------------+--------------------+--------------------+-------------------+
|  count|              170653|              170653|              170653|             170653|
|   mean|  0.5376399623706486|   533.2726662178331|  0.5288041167238217| 0.5021736129637326|
| stddev| 0.17595879531415606|  13500.997419182748|  0.2630074934143846|0.37601446946218486|
|    min| ""Kamehameha Sch...| ""Loughton High ...|                 0.0|                0.0|
|    max|               0.988|              974773|6.409999999999999...|            9.9e-06|
+-------+--------------------+--------------------+--------------------+-------------------+



In [14]:
audio_features = [
    "instrumentalness", "liveness", "speechiness", "tempo", "loudness"
]

df.select(audio_features).describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+-------------------+
|summary|    instrumentalness|            liveness|         speechiness|               tempo|           loudness|
+-------+--------------------+--------------------+--------------------+--------------------+-------------------+
|  count|              170653|              170653|              170653|              170653|             170653|
|   mean|   36.52932919284932|    3.57524000900989|   5.064387185348328|  118.22609112814219|-11.417017795610022|
| stddev|   3141.909337783477|     981.33368266444|   98.12127900967258|   67.00273009940202| 5.7298266024160736|
|    min| 'Benjamin Schrader'| 'Jason Michael S...| 'Allman Brothers...|    'José Feliciano'|   'David Baxter']"|
|    max|             9.9e-06|            9.53e-06|          You Did It|When Your Lover H...|                  9|
+-------+--------------------+--------------------+--------------------+----------------